# SPSC Tree Builder

Notebook để đọc file Excel (SPSC.xlsx) có các cột `Key`, `Parent key`, `Code`, `Title`, xây dựng cây phân cấp và lưu ra JSON (`spsc_data/spsc_tree.json`).

Chỉnh sửa đường dẫn/tên cột ở ô cấu hình bên dưới nếu khác mặc định.


In [1]:
import os
import json
from typing import Any, Dict, List, Optional

import pandas as pd

# Cấu hình đường dẫn (có thể chỉnh sửa)
repo_root = os.path.abspath(os.path.join(os.getcwd()))
input_excel_path = os.path.join(repo_root, "SPSC.xlsx")
output_json_path = os.path.join(repo_root, "spsc_data", "spsc_tree.json")

# Tên cột (có thể chỉnh sửa nếu file khác tên)
default_key_col = "Key"
default_parent_col = "Parent key"
default_code_col = "Code"
default_title_col = "Title"

print("Input Excel:", input_excel_path)
print("Output JSON:", output_json_path)


Input Excel: d:\HUST\.Lab\Product_Similarity\spsc_data\SPSC.xlsx
Output JSON: d:\HUST\.Lab\Product_Similarity\spsc_data\spsc_data\spsc_tree.json


In [2]:
def resolve_column_names(df: pd.DataFrame, desired: Dict[str, Optional[str]]) -> Dict[str, Optional[str]]:
    """Ánh xạ tên cột logic sang tên cột thật trong DataFrame (không phân biệt hoa/thường, loại khoảng trắng).

    desired: mapping logical -> provided column name (hoặc None nếu dùng mặc định).
    Trả về mapping logical -> tên cột thật (hoặc None nếu không tìm thấy/không cần).
    """
    normalized_to_actual: Dict[str, str] = {}
    for col in df.columns:
        normalized_to_actual[col.strip().lower()] = col

    resolved: Dict[str, Optional[str]] = {}
    for logical, provided in desired.items():
        if provided is None:
            key = logical.strip().lower()
            actual = normalized_to_actual.get(key)
            if actual is None:
                key2 = " ".join(key.split())
                for k, v in normalized_to_actual.items():
                    if " ".join(k.split()) == key2:
                        actual = v
                        break
            resolved[logical] = actual
        else:
            key = provided.strip().lower()
            resolved[logical] = normalized_to_actual.get(key, provided)
    return resolved


In [3]:
def normalize_key(value):
    """Chuẩn hóa giá trị khóa: loại bỏ đuôi '.0' khi là số nguyên, xử lý NaN/None.

    Trả về None nếu không có giá trị; ngược lại trả về chuỗi đã chuẩn hóa.
    """
    if value is None:
        return None
    try:
        import pandas as pd  # dùng trong notebook
        if pd.isna(value):
            return None
    except Exception:
        pass

    # Trường hợp kiểu số
    if isinstance(value, int):
        return str(value)
    if isinstance(value, float):
        if value.is_integer():
            return str(int(value))
        return str(value)

    s = str(value).strip()
    if s.endswith(".0"):
        s = s[:-2]
    return s


In [4]:
def build_tree_from_df(
    df: pd.DataFrame,
    key_col: str,
    parent_col: str,
    code_col: Optional[str],
    title_col: Optional[str],
) -> List[Dict[str, Any]]:
    nodes_by_key: Dict[str, Dict[str, Any]] = {}
    children_of_parent: Dict[str, List[str]] = {}

    for _, row in df.iterrows():
        # Chuẩn hóa key, loại bỏ đuôi .0 nếu có
        key = normalize_key(row.get(key_col))
        if not key:
            continue

        # Chuẩn hóa parent key
        parent_key = normalize_key(row.get(parent_col))

        code: Optional[str] = None
        if code_col is not None and code_col in df.columns:
            val = row.get(code_col)
            if val is not None and not pd.isna(val):
                code = str(val).strip()

        title: Optional[str] = None
        if title_col is not None and title_col in df.columns:
            val = row.get(title_col)
            if val is not None and not pd.isna(val):
                title = str(val).strip()

        node = nodes_by_key.get(key)
        if node is None:
            node = {"key": key, "code": code, "title": title, "children": []}
            nodes_by_key[key] = node
        else:
            if node.get("code") is None and code is not None:
                node["code"] = code
            if node.get("title") is None and title is not None:
                node["title"] = title

        if parent_key:
            children_of_parent.setdefault(parent_key, []).append(key)
            node["_parent_key"] = parent_key

    # Ensure all referenced parents exist as nodes
    for parent_key in list(children_of_parent.keys()):
        if parent_key not in nodes_by_key:
            nodes_by_key[parent_key] = {"key": parent_key, "code": None, "title": None, "children": []}

    # Attach children to parents
    for parent_key, child_keys in children_of_parent.items():
        parent_node = nodes_by_key[parent_key]
        for child_key in child_keys:
            child_node = nodes_by_key[child_key]
            parent_node["children"].append(child_node)

    roots: List[Dict[str, Any]] = [n for n in nodes_by_key.values() if "_parent_key" not in n]

    # Cleanup temp fields
    for n in nodes_by_key.values():
        if "_parent_key" in n:
            del n["_parent_key"]

    return roots


In [5]:
# Đọc dữ liệu từ Excel
try:
    df = pd.read_excel(input_excel_path)
except Exception as exc:
    raise RuntimeError(f"Không đọc được file Excel: {exc}")

print("Số dòng:", len(df))
df.head(3)


Số dòng: 13293


,Key,Parent key,Code,Title
0,100,NaN,A,"Raw Materials, Chemicals, Paper, Fuel"
1,101,NaN,B,Industrial Equipment & Tools
2,102,NaN,C,Components & Supplies


In [6]:
# Ánh xạ tên cột
resolved = resolve_column_names(
    df,
    {
        "Key": default_key_col,
        "Parent key": default_parent_col,
        "Code": default_code_col,
        "Title": default_title_col,
    },
)

key_col = resolved.get("Key")
parent_col = resolved.get("Parent key")
code_col = resolved.get("Code")
title_col = resolved.get("Title")

print("Đã xác định cột:", resolved)

missing = []
if key_col is None or key_col not in df.columns:
    missing.append("Key")
if parent_col is None or parent_col not in df.columns:
    missing.append("Parent key")
if missing:
    raise ValueError(f"Thiếu cột bắt buộc: {', '.join(missing)}")


Đã xác định cột: {'Key': 'Key', 'Parent key': 'Parent key', 'Code': 'Code', 'Title': 'Title'}


In [7]:
# Xây cây
roots = build_tree_from_df(df, key_col, parent_col, code_col, title_col)
print(f"Số lượng node gốc: {len(roots)}")
# Hiển thị keys của node gốc đầu tiên (nếu có)
if roots:
    list(roots[0].keys())


Số lượng node gốc: 18


In [8]:
# Lưu JSON và xem trước một phần
os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump({
        "roots": roots,
        "meta": {
            "input": input_excel_path,
            "columns": {
                "key": key_col,
                "parent": parent_col,
                "code": code_col,
                "title": title_col,
            },
            "num_roots": len(roots),
        },
    }, f, ensure_ascii=False, indent=2)

print("Đã ghi:", output_json_path)
# Xem trước một node gốc đầu tiên (nếu có)
if roots:
    from pprint import pprint
    sample = roots[0].copy()
    # Ẩn bớt children để xem nhanh
    sample_children = sample.get("children", [])
    sample["children"] = f"{len(sample_children)} children (ẩn)"
    pprint(sample)


Đã ghi: d:\HUST\.Lab\Product_Similarity\spsc_data\spsc_data\spsc_tree.json
{'children': '6 children (ẩn)',
 'code': 'A',
 'key': '100',
 'title': 'Raw Materials, Chemicals, Paper, Fuel'}
